In [1]:
gazeta_test = "/home/alex/dev/AiLearning/DataSetStore/text_summarization/gazeta_test.txt"
gazeta_train = "/home/alex/dev/AiLearning/DataSetStore/text_summarization/gazeta_train.txt"
gazeta_valid = "/home/alex/dev/AiLearning/DataSetStore/text_summarization/gazeta_val.txt"

1. <b>Reading</b>

In [6]:
from tqdm.notebook import tqdm
import json

def read_lines(input_path:str, callback=lambda a: a, limit=None, verbose=False):
    collector = list()
    line_cnt = 0
    with open(input_path,"r") as reader:
        for line in tqdm(reader):
            line = callback(line)
            collector.append(line)
            line_cnt=line_cnt+1
            if verbose:
                print ("Parsed: ",line)
            if limit is not None and line_cnt>=limit:
                break
    return collector

def parse_callback (text):
    return json.loads(text)

<b>Testing</b>

In [8]:
print ("Test-case reading:",read_lines(input_path=gazeta_train,callback=parse_callback,limit=2,verbose=True))

0it [00:00, ?it/s]

Parsed:  {'url': 'https://www.gazeta.ru/financial/2011/11/30/3852658.shtml', 'text': '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означ

2. <b>Metrics</b>

In [11]:
!pip install rouge


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [12]:
from rouge import Rouge

def calc_scores(valid_summary, predicted_summary, verbose=False):
    rouge = Rouge()
    scores = rouge.get_scores(valid_summary, predicted_summary, avg=True)
    if verbose:
        print("Rouge: ", scores)
        for v,p in zip(valid_summary,predicted_summary):
            print ("valid={} vs pred={}".format(v,p))
    return scores


<b>Testing</b>

In [31]:
print ("Test-case 1:",calc_scores("I am going at school", "I am going to the stadium"))
print ("Test-case 2:",calc_scores("I am going at school", "I am swimming now"))
print ("Test-case 3:",calc_scores(["I am going at school","I am going at school"], ["I am going to the stadium","I am swimming now"]))

Test-case 1: {'rouge-1': {'r': 0.5, 'p': 0.6, 'f': 0.5454545404958678}, 'rouge-2': {'r': 0.4, 'p': 0.5, 'f': 0.4444444395061729}, 'rouge-l': {'r': 0.5, 'p': 0.6, 'f': 0.5454545404958678}}
Test-case 2: {'rouge-1': {'r': 0.5, 'p': 0.4, 'f': 0.4444444395061729}, 'rouge-2': {'r': 0.3333333333333333, 'p': 0.25, 'f': 0.2857142808163266}, 'rouge-l': {'r': 0.5, 'p': 0.4, 'f': 0.4444444395061729}}
Test-case 3: {'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4949494900010204}, 'rouge-2': {'r': 0.3666666666666667, 'p': 0.375, 'f': 0.36507936016124976}, 'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4949494900010204}}


<b>Additional</b>

In [34]:
train_dict_list = read_lines(input_path=gazeta_train,
                             callback=parse_callback,
                             limit=300,
                             verbose=False)

def split_text_and_summary(dict_list: list):
    text_list = list()
    summary_list = list()
    for dictx in dict_list:
        text_list.append(dictx['text'].lower())
        summary_list.append(dictx['summary'].lower())
    return text_list,summary_list

train_text_list,train_summary_list = split_text_and_summary(dict_list=train_dict_list)


0it [00:00, ?it/s]

In [35]:
from abc import ABC
import razdel
class AbstractSummarizer(ABC):
    def fit_transform(self, train_text_list: list, valid_text_list: list) -> list:
        ...
    def transform(self, valid_text_list: list) -> list:
        ...

3. <b>Top -N</b>

In [41]:
import razdel

class TopN (AbstractSummarizer):
    def __init__(self,n):
        self.n=n
    
    def fit_transform(self, train_text_list: list, valid_text_list: list):
        return self.transform(valid_text_list=valid_text_list)

    def transform(self,valid_text_list: list) -> list:
        predicted_summary_list=list()
        for text in valid_text_list:
            sentences = [sentence.text for sentence in razdel.sentenize(text)]
            predicted_summary_list.append(" ".join(sentences[:self.n]))
        return predicted_summary_list



<b>Testing</b>

In [43]:
#train_text_list

for i in range(1,5):
    method = TopN(n=i)
    predictions = method.transform(valid_text_list=train_text_list)
    
    print( i, calc_scores(valid_summary=train_summary_list, predicted_summary=predictions))


1 {'rouge-1': {'r': 0.15841581392483228, 'p': 0.34848636543905565, 'f': 0.18126678787378214}, 'rouge-2': {'r': 0.05883170479440949, 'p': 0.11845646806531475, 'f': 0.06311124154784647}, 'rouge-l': {'r': 0.14468629953847723, 'p': 0.32137857736910497, 'f': 0.16597067159173293}}
2 {'rouge-1': {'r': 0.09171913037931169, 'p': 0.4526151881867905, 'f': 0.14429530821541986}, 'rouge-2': {'r': 0.0280016887619996, 'p': 0.15660763494451047, 'f': 0.04466371187860624}, 'rouge-l': {'r': 0.08504560513642541, 'p': 0.4221003478842912, 'f': 0.1340263118962767}}
3 {'rouge-1': {'r': 0.07283251152763322, 'p': 0.5034648911043871, 'f': 0.12340528185001455}, 'rouge-2': {'r': 0.02119930536565555, 'p': 0.17651435966424647, 'f': 0.036653666076025174}, 'rouge-l': {'r': 0.06839455679444816, 'p': 0.4730425903604309, 'f': 0.11588167812379598}}
4 {'rouge-1': {'r': 0.06332865102033128, 'p': 0.5315279865246064, 'f': 0.11122330825436094}, 'rouge-2': {'r': 0.017753348824046798, 'p': 0.1867668881128964, 'f': 0.0318813381510